In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import captum

import torch
import torchtext
import torchtext.data

import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
from torchtext.data.utils import get_tokenizer

from models.lstm.model import LSTM
from datasets.scar import SCAR


tokenizer = get_tokenizer('basic_english')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#Survival 60 mo
ckpt_path = os.path.join(r"C:\Users\jjnunez\PycharmProjects\scar_nlp\results\survic_mo_60\LSTM", "LSTM_20220222-1640.pt")

checkpoint = torch.load(ckpt_path)

model = LSTM(config=checkpoint['config'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model = model.to(device)
print(model)

self.embed num_embeddings is: 17411 while embedding_dim is 300
Applying weight drop of 0.2 to weight_hh_l0
LSTM(
  (embed): Embedding(17411, 300)
  (lstm): WeightDrop(
    (module): LSTM(300, 512, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1024, out_features=1, bias=True)
)


C:\Users\jjnunez\.conda\envs\scar_nlp\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [4]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

In [5]:
# TEXT = torchtext.legacy.data.Field(lower=True, tokenize=tokenizer)
# Label = torchtext.legacy.data.LabelField(dtype = torch.float)

In [6]:
# train = IMDB(split='train')
#test = IMDB(split='test')

In [7]:
# from torchtext import vocab

#loaded_vectors = vocab.GloVe(name='6B', dim=100)

#TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
    
#TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
#Label.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))

#print('Vocabulary Size: ', len(TEXT.vocab))
#print('Vocabulary Size: ', len(Label.vocab))
#print(train)
#print(Label.vocab.itos)
config = checkpoint['config']
scar = SCAR(config.batch_size, config.data_dir, config.target)



In [8]:
vocab = scar.vocab
itos = vocab.get_itos()
print(len(vocab))

17411


In [19]:
# PAD_IND = TEXT.vocab.stoi[TEXT.pad_token]
print(vocab['doctor'])
PAD_IND = vocab['<PAD>']
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embed)
print(lig)

884


In [17]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []
lig = LayerIntegratedGradients(model, model.embed)
def interpret_sentence(model, sentence, min_len = 50, label = 0):
    text = [tok for tok in tokenizer(sentence.lower())]
    if len(text) < min_len:
        text += ['<PAD>'] * (min_len - len(text))
    indexed = [vocab[t] for t in text]

    model.zero_grad()
   
    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)
    
    print(f'here is: {pred}')

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
    
    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=500, return_convergence_delta=True)
    # Replace Label with Text below
    print(itos[pred_ind])
    print('pred: ', itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            itos[pred_ind],
                            itos[label],
                            itos[1],
                            attributions.sum(),
                            text,
                            delta))

In [18]:
text = "Ms. Jones is a 64 year old woman with metastatic stage 4 lung cancer. She lives alone and does not work. She feels very sad. She has a lot of pain."
# text = "Ms. Jones is a 64 year old woman with metastatic stage 4 lung cancer."
interpret_sentence(model, text, label=0)



here is: 0.3750191330909729


AssertionError: Forward hook did not obtain any outputs for given layer

In [ ]:
print('Visualize attributions based on Integrated Gradients')
_ = visualization.visualize_text(vis_data_records_ig)